<a href="https://colab.research.google.com/github/2ndt2nd/Final-Project-IAT-360/blob/main/Final_Project_Jacques_Seth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b> IAT 360 Final Project <b> </h1></center>

This is the final project for IAT 360
Group Members:
    - Seth van der Zweep (301461888)
    - Jacques Davidson Widodo (301443163)


<h2>Importing required Libraries<h2>

<strong>Import Ultralytics</strong>

Run this one


In [17]:
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

model = YOLO('best.pt')

#CV2 as well
import cv2
cap = cv2.VideoCapture(0)  # Try different indices like 0, 1, 2
if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("Webcam opened successfully!")
    
cap.release()

Ultralytics 8.3.38 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Setup complete ✅ (8 CPUs, 19.3 GB RAM, 0.0/9.7 GB disk)
Webcam opened successfully!


<strong>Enum States</strong>
For the states

Run this one


In [18]:
from enum import Enum

class State(Enum):
    START = 0
    SELECTING_ROW = 1
    SELECTING_COLUMN = 2
    CONFIRMING = 3
    DISPENSING = 4

reorder = [3,7,2,5,6,1,4,8]

<h2>New In Progress Video Function</h2>

In [19]:
def new_process_video_feed(model, cap):
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("'''Error: Failed to grab frame.")
            return 0

        frame = cv2.resize(frame, (640, 480))
        results =   model.predict(frame, conf=0.7)  # YOLO inference
        rendered_frame = results[0].plot()
        cv2.imshow("Webcam Feed", rendered_frame)

        class_ids = [int(box.cls[0].item()) for box in results[0].boxes]

        if(not class_ids): return class_ids
        return reorder[class_ids[0]]

<h2>New In Progress Main</h2>

In [ ]:
def main(model):
    hold = 0
    dispensing_timer = 0
    state = State.START

    row = 0
    column = 0

    #Images section
    #Two Dimensional Array after finishing column selection
    starting_image = 'images/select_start.png'
    selection_image = 'images/select_row.png' #Used for state SELECTING_ROW
    rows_option = ['images/row_a.png','images/row_b.png','images/row_c.png','images/row_d.png'] #Used for state SELECTING_COLUMN
    
    #Used for state CONFIRMING
    row_col_option = [
        #Row A
        ['images/a1.png','images/a2.png','images/a3.png','images/a4.png'],
        #Row B
        ['images/b1.png','images/b2.png','images/b3.png','images/b4.png'],
        #Row C
        ['images/c1.png','images/c2.png','images/c3.png','images/c4.png'],
        #Row D
        ['images/d1.png','images/d2.png','images/d3.png','images/d4.png']
        ]
    
    #Used for state DISPENSING
    row_col_out = [
        #Row A
        ['images/disp_a1.png','images/disp_a2.png','images/disp_a3.png','images/disp_a4.png'],
        #Row B
        ['images/disp_b1.png','images/disp_b2.png','images/disp_b3.png','images/disp_b4.png'],
        #Row C
        ['images/disp_c1.png','images/disp_c2.png','images/disp_c3.png','images/disp_c4.png'],
        #Row D
        ['images/disp_d1.png','images/disp_d2.png','images/disp_d3.png','images/disp_d4.png']
        ]
    current_image = cv2.imread(starting_image)
    current_image = cv2.resize(current_image, (793, 848))
    cv2.imshow('Dispenser', current_image)
    
    #Video handling
    #cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # For Windows
    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # For Linux
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
    cap.set(cv2.CAP_PROP_FPS, 30)  # Adjust FPS
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    #Main Loop
    while True:
        gesture = new_process_video_feed(model, cap)
        print(state)
        print(state.value)
        
        if(dispensing_timer > 0):
            dispensing_timer -= 1
            if(dispensing_timer == 0):
                state = State.START
                current_image = cv2.imread(starting_image)
                current_image = cv2.resize(current_image, (793, 848))
                cv2.imshow('Dispenser', current_image)

        #When no gestures are detected
        if(not gesture):
            hold = 0
        else:
            #If condition to prevent accidentally proceeding
            if(hold<15): hold += 1
            print(gesture)
        
        #Until user shows no gesture by making fist, cannot make another command.
        if(hold == 10):
            hold = 100

            #For start and confirmation
            if(state == State.START):
                if(gesture == 8):
                    state = State(state.value+1)
                    current_image = cv2.imread(selection_image)
            
            #between selecting rows and before dispensing
            elif(state.value>0 and state.value<4):
                #For numerical gestures
                if(gesture<5 and state.value<3):
                    if(state == State.SELECTING_ROW):
                        row = gesture
                        current_image = cv2.imread(rows_option[row-1])


                    elif(state == State.SELECTING_COLUMN):
                        column = gesture
                        current_image = cv2.imread(row_col_option[row-1][column-1])
                    
                    state = State(state.value+1)

                #Gesture thumb_index
                elif(gesture == 7):
                    column = 0
                    row = 0
                    state = State.START
                    current_image = cv2.imread(starting_image)
                
                #Gesture dislike
                elif (gesture == 6):
                        if(state == State.SELECTING_ROW):
                            row = 0
                            current_image = cv2.imread(starting_image)
                            

                        elif(state == State.SELECTING_COLUMN):
                            column = 0
                            current_image = cv2.imread(selection_image)
                        
                        elif(state == State.CONFIRMING):
                            column = 0
                            current_image = cv2.imread(rows_option[row-1])
                        
                        state = State(state.value-1)
                
                elif(gesture == 8 and state == State.CONFIRMING):
                    state = State(state.value+1)
                    current_image = cv2.imread(row_col_out[row-1][column-1])
                    dispensing_timer = 15

        current_image = cv2.resize(current_image, (793, 848))
        cv2.imshow('Dispenser', current_image)
        

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

<h2>Old Main</h2>
Run this one

In [21]:
main(model)


0: 480x640 (no detections), 115.0ms
Speed: 1.0ms preprocess, 115.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
State.START
0

0: 480x640 (no detections), 121.1ms
Speed: 1.0ms preprocess, 121.1ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
State.START
0

0: 480x640 (no detections), 113.9ms
Speed: 1.2ms preprocess, 113.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
State.START
0

0: 480x640 (no detections), 109.2ms
Speed: 3.3ms preprocess, 109.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
State.START
0

0: 480x640 (no detections), 113.4ms
Speed: 1.3ms preprocess, 113.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
State.START
0

0: 480x640 (no detections), 114.1ms
Speed: 1.2ms preprocess, 114.1ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)
State.START
0

0: 480x640 (no detections), 107.0ms
Speed: 1.2ms preprocess, 107.0ms inference, 0.3ms postprocess per ima

<strong>For Linux</strong>

Testing video output for Linux (different back end)

<h2>Testing Video Only</h2>

In [22]:
'''def process_video_feed(model):
    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # Adjust backend for your OS
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)
    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Error: Failed to grab frame.")
            break

        frame = cv2.resize(frame, (640, 480))
        results = model.predict(frame, conf=0.75)  # YOLO inference
        rendered_frame = results[0].plot()
        cv2.imshow("Webcam Feed", rendered_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()'''

'def process_video_feed(model):\n    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # Adjust backend for your OS\n    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)\n    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)\n    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS\n    \n    if not cap.isOpened():\n        print("Error: Could not open webcam.")\n        return\n\n    while True:\n        ret, frame = cap.read()  # Capture frame\n        if not ret:\n            print("Error: Failed to grab frame.")\n            break\n\n        frame = cv2.resize(frame, (640, 480))\n        results = model.predict(frame, conf=0.75)  # YOLO inference\n        rendered_frame = results[0].plot()\n        cv2.imshow("Webcam Feed", rendered_frame)\n\n        if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n            break\n    \n    cap.release()\n    cv2.destroyAllWindows()'

<strong>For Windows</strong>

Testing video output for windows (different back end)

In [23]:
'''def process_video_feed(model):
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Adjust backend for your OS
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)
    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Error: Failed to grab frame.")
            break

        frame = cv2.resize(frame, (640, 480))

        results = model.predict(frame, conf=0.75)  # YOLO inference
        rendered_frame = results[0].plot()
        cv2.imshow("Webcam Feed", rendered_frame)

        class_ids = [int(box.cls[0].item()) for box in results[0].boxes]
        print(class_ids)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    
    
    cap.release()
    cv2.destroyAllWindows()'''

'def process_video_feed(model):\n    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Adjust backend for your OS\n    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)\n    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)\n    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS\n    \n    if not cap.isOpened():\n        print("Error: Could not open webcam.")\n        return\n\n    while True:\n        ret, frame = cap.read()  # Capture frame\n        if not ret:\n            print("Error: Failed to grab frame.")\n            break\n\n        frame = cv2.resize(frame, (640, 480))\n\n        results = model.predict(frame, conf=0.75)  # YOLO inference\n        rendered_frame = results[0].plot()\n        cv2.imshow("Webcam Feed", rendered_frame)\n\n        class_ids = [int(box.cls[0].item()) for box in results[0].boxes]\n        print(class_ids)\n\n        if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n            break\n    \n    \n    \n    cap.release()\n    cv2.destroyAllWindows()'

In [24]:
# Start processing the webcam feed
#process_video_feed(model)
#main(model)
